<a href="https://colab.research.google.com/github/heberianmani/ApacheBeam_Workouts/blob/main/Streaming_Data_Pipelines_Beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!ls 

sample_data


In [14]:
from google.colab import files
uploaded = files.upload()

Saving counts.csv to counts.csv


In [8]:
from apache_beam import coders
coders.registry.get_coder(int)
#coders.registry.register_coder(int,coders.FloatCoder)

FloatCoder

In [9]:
import apache_beam as beam

p = beam.Pipeline()

evens = ( p 
         | beam.Create(['one','two','three']) 
         | beam.Filter(lambda x: x % 2 == 0).with_input_types(int) 
        )
  
p.run()

TypeCheckError: ignored

In [10]:
# import beam module
import apache_beam as beam

p = beam.Pipeline()

@beam.typehints.with_input_types(int)
class FilterEvensDoFn(beam.DoFn):
  def process(self, element):
    if element % 2 == 0:
      yield element

evens = ( p
         | beam.Create(['1','2','3'])
         | beam.ParDo(FilterEvensDoFn()) 
        )
  
p.run()

TypeCheckError: ignored

In [13]:
import apache_beam as beam
import typing

class Employee(object):
  def __init__(self, id, name):
    self.id = id
    self.name = name

class EmployeeCoder(beam.coders.Coder):

  def encode(self, employee):
    return ('%s:%s' % (employee.id, employee.name)).encode('utf-8')

  def decode(self, s):
    return Employee(*s.decode('utf-8').split(':'))

  def is_deterministic(self):
    return True

beam.coders.registry.register_coder(Employee, EmployeeCoder)

def split_file(input):
  name, id, salary = input.split(',')
  return Employee(id, name), int(salary)

result = (
    p
	| beam.io.ReadFromText('data.txt')
    | beam.Map(split_file)
    | beam.CombinePerKey(sum).with_input_types(typing.Tuple[Employee, int])
	)
	
p.run()	

ValueError: ignored